note - there are 2 cells that need adjusting depending on which set (small sample, or real set) is being used:
cell 73 & cell 376

ISSUES
 - if the video is unavailable, the scrape part of the algo returns a web address with just no track name. It needs to search again.

## import & wrangling

In [1]:
import pandas as pd
import numpy as np
import random
import time
from bs4 import BeautifulSoup as bs
import requests

In [65]:
path_to_file = "/Users/aidanairuser/Desktop/OPENWHYHD/openwhyhd_logs.csv"
sample = pd.read_csv(path_to_file)
print (sample.shape)
sample.sample(5)

(584927, 2)


,user,song
472088,47,NP57BsWO3RY
166836,155,FKIQ8BBcI6I
490674,1000,xuTDKlyXbQg
279987,285,fbzEOQLOAWw
231042,285,pguMUFyJ3_U


In [66]:
print ("nb of songs", sample.user.nunique())
print ("nb of users", sample.song.nunique())
print ("DATA TYPES")
print (sample.dtypes)

# there are 96 users in the sample set

nb of songs 1275
nb of users 80388
DATA TYPES
user     int64
song    object
dtype: object


## simplest rec: a top ten

In [105]:
top_ten = sample.song.value_counts().head(11)
hits = pd.DataFrame(top_ten)
hits

,song
x1ea7Dh53lg,4226
y3FGucaZk2Y,4219
pguMUFyJ3_U,4204
MwvITR0d93g,4193
Y9GKNbNSpM8,4178
g_XNPNYCUSU,4178
cLbgNvjyS4I,4176
B42HTy-gRVA,4172
fbzEOQLOAWw,4171
OmEyb87NlQo,4167


-- this is the top ten for the small sample set

scrape the details (see below):

少年時代　井上揚水
Green Velvet - I Want To Leave My Body
般若心経 托鉢 short ver  (heartsutra takuhatsu ver.) live act / 薬師寺寛邦(キッサコ)
般若心経 cho ver. (Heartsutra cho ver. ) live act / 薬師寺寛邦 キッサコ
BEACH HOUSE -- "LEMON GLOW"
Rejjie Snow - Egyptian Luvr (feat. Aminé & Dana Williams) (Official Audio)
The Spinners - I'll Be Around
** the 8th most listened to is now 'video unavailable
6LACK - Never Know
Best Music Mix 2017- Shuffle Music Video HD - Melbourne Bounce Music Mix 2017
Underground Lovers - Your Eyes

## function to pull video title from site

In [106]:

def find_toon(rem):
    
    top_ten_listens = []
    
    yt = "https://www.youtube.com/watch?v="
    url = yt.strip() + rem.strip()
    page = requests.get(url)
    soup = bs (page.text, 'html.parser')

    for item in soup.find_all('h1',{'class':'watch-title-container'}): 
        for post in item.find_all('span',{'class':'watch-title'}): 
            print (post.string.strip())
            print (url)
            
find_toon("PLACE_end_of_yt_address_here_please")

The Shadow of Your Smile - Perry Como Live
https://www.youtube.com/watch?v=x1ea7Dh53lg


## pivot and flatten the log

In [67]:
# flip sample to a multi-index / pivot table (aggfunc=len and aggfunc='count' both work as arguments)

freq = pd.pivot_table(sample,index=["song"],columns=["user"],values=["user"], aggfunc={"user":'count'})

freq.head(5)

user                                              ...             \
user         0    1    2    3    4    5    6    7    8    9    ...  1265 1266   
song                                                           ...              
 --1W3Kq_pfk  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --4Xde9r4Mw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --5UoWHFEpw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --ASOotWk1E  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --BOTvEjuOg  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0 ...   NaN  NaN   

                                                      
user         1267 1268 1269 1270 1271 1272 1273 1274  
song                                                  
 --1W3Kq_pfk  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --4Xde9r4Mw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --5UoWHFEpw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --ASOotWk1E  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --BOTvEjuOg  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 1275 columns]

In [68]:
# turn selection into a df (multi-index)

pivoted = pd.DataFrame(freq)

print (pivoted.shape)

pivoted.head(5)

(80388, 1275)


user                                              ...             \
user         0    1    2    3    4    5    6    7    8    9    ...  1265 1266   
song                                                           ...              
 --1W3Kq_pfk  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --4Xde9r4Mw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --5UoWHFEpw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --ASOotWk1E  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
 --BOTvEjuOg  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0 ...   NaN  NaN   

                                                      
user         1267 1268 1269 1270 1271 1272 1273 1274  
song                                                  
 --1W3Kq_pfk  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --4Xde9r4Mw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --5UoWHFEpw  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --ASOotWk1E  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 --BOTvEjuOg  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 1275 columns]

In [73]:
# how sparse is the df? Print the total number of NaN and give as a %

count_nans = pivoted.isnull().sum().sum()
total_nb_cells = 80388 * 1275 
# total_nb_cells = 1688 * 96 # this line is for when we're using the sample set
perc = (count_nans/total_nb_cells)*100

print (count_nans, "cells are empty - soit", perc.round(decimals=2), "%", "of the df")


102324954 cells are empty - soit 99.83 % of the df


In [74]:
####### ------------ RELOAD FROM HERE IF NECESSARY

# remove a dimension to get a df with regular columns

pivoted.columns = pivoted.columns.get_level_values(1)
pivoted_done = pd.DataFrame(pivoted)

In [75]:
type(pivoted_done)

pandas.core.frame.DataFrame

In [76]:
pivoted_done.columns

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274],
           dtype='int64', name='user', length=1275)

In [77]:
# replace the NaN with 0

pivoted_done.fillna(value = 0, inplace = True)

In [101]:
# change column names from ints to "U6" etc.

string = "U"
pivoted_done.columns = [string+str(i) for i in range(1275)]


Index(['U0', 'U1', 'U2', 'U3', 'U4', 'U5', 'U6', 'U7', 'U8', 'U9',
       ...
       'U1265', 'U1266', 'U1267', 'U1268', 'U1269', 'U1270', 'U1271', 'U1272',
       'U1273', 'U1274'],
      dtype='object', length=1275)

In [102]:
# cast all entries to integers, not floats

pivoted_int = pivoted_done.astype(int, inplace = True)
pivoted_int.dtypes.head(5)

# add (int, errors = 'ignore') if that's what you need


U0    int64
U1    int64
U2    int64
U3    int64
U4    int64
dtype: object

In [103]:
# THIS returns ONLY the integer - slice the row, then slice the column

pivoted_int.iloc[0]["U1"]


0

In [104]:
pivoted_int.shape

(80388, 1275)

## FINAL ALGORITHM (soir, mardi 19 mars)

In [376]:

def filled_test(user_A, user_B):

    print ("The pair under consideration are", user_A, "and", user_B, "\n")

    # establish whether there are matches between the 2 users
    match_count_df = pivoted_int[[user_A, user_B]]
    a_b_match_count = len (match_count_df[(match_count_df[user_A] != 0) & (match_count_df[user_B] != 0)])

    if a_b_match_count == 0:
        print ("These two users have never listened to the same song, so they have nothing to suggest to the other", "\n")
    
    else:
    
        # cell is a variable so that we can increase the row by one each time
        row = 0
    
        # declare two empty lists, to store offers from one user to another
        b_TO_A_offer = []
        a_TO_B_offer = []

        # for x in range (1688): FOR USE WITH SMALL SAMPLE SET 
        for x in range (80388):
                  
            # find a song present in right, but not left, to be offered
            if pivoted_int.iloc[row][user_B] > 0 and pivoted_int.iloc[row][user_A] == 0:
                b_TO_A_offer += [row]
            
            # find a song present in left col, but not right, to be offered
            if pivoted_int.iloc[row][user_A] > 0 and pivoted_int.iloc[row][user_B] == 0:
                a_TO_B_offer += [row]
        
            row = row + 1
    
        print (a_b_match_count, "match(es) in total: Strength of match is TO BE DEFINED HERE PLEASE IN PART THROUGH % MATCH, IN PART THROUGH THE NB OF LISTENS TO EACH SONG \n")

        print (len (b_TO_A_offer), "offers for A col:", b_TO_A_offer, "\n")
        print (len (a_TO_B_offer), "offers for B col:", a_TO_B_offer, "\n")
 
        # pick from the list -- at random -- and make an offer for user A
        songA = random.choice(b_TO_A_offer)
        song_address = pivoted_int.index[songA]
        yt = "https://www.youtube.com/watch?v="
        completeA = yt.strip() + song_address.strip()
        print(user_A, "might like", completeA)
                      
        # pull video details from youtube for suggestion to user A - NOTE: 4 TIMES AS SLOW WITH THE TWO SCRAPES
        url = completeA
        page = requests.get(url)
        soup = bs (page.text, 'html.parser')
        for item in soup.find_all('h1',{'class':'watch-title-container'}): 
            for post in item.find_all('span',{'class':'watch-title'}): 
                details = post.string.strip()
                print (details, "\n")
    
        #  ... and for user B
        songB = random.choice(a_TO_B_offer)
        song_address = pivoted_int.index[songB]
        yt = "https://www.youtube.com/watch?v="
        completeB = yt.strip() + song_address.strip()
        print(user_B, "might like", completeB)
               
        # pull video details from youtube for suggestion to user B
        url = completeB
        page = requests.get(url)
        soup = bs (page.text, 'html.parser')
        for item in soup.find_all('h1',{'class':'watch-title-container'}): 
            for post in item.find_all('span',{'class':'watch-title'}): 
                details = post.string.strip()
                print (details, "\n")
    
    print ("HERE WE NEED TO FIND A WAY OF STORING UP THE DIFFERENT 'OFFERED TO U41' SERIES SO THAT WE CAN LOOK AT ALL THE SONGS BEING OFFERED TO B BY THE OTHER 95 USERS, AND CHOOSE THE MOST OFFERED", "\n")

    # this has been added to calculate the average nb of plays for each song a user listens to
    no_zeros_A = pivoted_int[(pivoted_int[user_A] != 0)] # all the full cells for the user column
    no_zeros_B = pivoted_int[(pivoted_int[user_B] != 0)]

    av_list_A = no_zeros_A[user_A].sum()/no_zeros_A[user_A].count() # a count (nb of songs) and a sum (nb of listens)
    av_list_B = no_zeros_B[user_B].sum()/no_zeros_B[user_B].count()

    print (user_A, "listened to each of his / her songs an average of", av_list_A.round(decimals=2), "times")
    print (user_B, "listened to each of his / her songs an average of", av_list_B.round(decimals=2), "times", "\n")

## run algo here:

In [377]:
start_time = time.time()

# insert HERE the two users to be compared
filled_test("U2", "U3")

print("RAN IN --- %s seconds ---" % (time.time() - start_time))

The pair under consideration are U2 and U3 

46 match(es) in total: Strength of match is TO BE DEFINED HERE PLEASE IN PART THROUGH % MATCH, IN PART THROUGH THE NB OF LISTENS TO EACH SONG 

717 offers for A col: [8, 136, 249, 256, 287, 412, 419, 471, 473, 625, 636, 662, 728, 771, 931, 1088, 1172, 1257, 1292, 1312, 1360, 1375, 1450, 1712, 1786, 1969, 2061, 2077, 2100, 2357, 2393, 2483, 2513, 2539, 2578, 2939, 2944, 3149, 3213, 3271, 3399, 3534, 3747, 3872, 3875, 3901, 3999, 4001, 4177, 4293, 4322, 4356, 4367, 4443, 4778, 4786, 4883, 5039, 5074, 5176, 5196, 5282, 5340, 5465, 5474, 5610, 5724, 5752, 5758, 5840, 5920, 6044, 6119, 6245, 6477, 6575, 6623, 6838, 6916, 6989, 7341, 7387, 7425, 7478, 7627, 7674, 7810, 8016, 8472, 8545, 8640, 9033, 9573, 9577, 9627, 9694, 9765, 9791, 9809, 9890, 10010, 10151, 10292, 10327, 10363, 10397, 10401, 10616, 10826, 10886, 10974, 11093, 11337, 11381, 11505, 11670, 11754, 11792, 11918, 11983, 11984, 12051, 12076, 12156, 12172, 12276, 12667, 12715, 12882, 13

## integrating the NUMBER of listens to a given song by a user to: 1 get a better picture of the strength of a match & 2 help get stronger recs

In [374]:
# How do the users compare in the number of different songs they've listened to?

# no, I think this is total listens

pivoted_int.astype(bool).sum(axis=0).head(5)

U0     234
U1    3134
U2     312
U3     763
U4     458
dtype: int64

In [315]:
# find number of non-zero cells in a user's column, i.e. the number of songs they've listened to

print (pivoted_int.U26.astype(bool).sum(axis=0), "songs played by this user")

15 songs played by this user


In [309]:
# show all songs played, with number of listens for each

user_A = "U26"

non_zeros = (pivoted_int[(pivoted_int[user_A] != 0) ])

non_zeros[user_A]


song
 1UIwqUkSnIc    2
 ABagIZHEWyo    2
 EFJ7kDva7JE    2
 LiUjOFYDiBE    1
 MC44L5T0FVA    2
 OM_WUDxhYps    4
 QxHkLdQy5f0    2
 RT__tPTEVVA    1
 RjLXOw414DA    3
 WKnVaDwUg5s    1
 ea2WoUtbzuw    1
 l8wWa3O9cUo    1
 n2rVnRwW0h8    1
 r-lDqAlXZHo    1
 z_mdSEuZ7k8    1
Name: U26, dtype: int64